In [67]:
import pandas as pd
from pulp import *
PATH = 'data/'

current_df = pd.read_csv(PATH + '2021-22/gws/merged_gw.csv', index_col=False)
gw_35 = current_df[current_df.GW == 35]


GW_dict = {}
GWs = current_df.GW.unique()
             
# Solving for the first x weeks, first part before free hit
players = current_df[current_df.GW == 1].name.unique()
for GW in sorted(GWs)[:35]:
    print(GW)
    player_price_dict = {} # key: player, value: (name, team, position, price, total points)
    print("Number players in week {} is {}".format(GW, len(players)))
    for i, player in enumerate(players):
        player_entries = current_df[current_df.name == player]
        total_points = player_entries[player_entries.GW <= GW]['total_points'].sum()
        player_team = player_entries[player_entries.GW == 1]['team'].values[0]
        player_price = player_entries[player_entries.GW == 1]['value'].values[0]
        player_position = player_entries[player_entries.GW == 1]['position'].values[0]
        player_name = player_entries[player_entries.GW == 1]['name']
        if player not in player_price_dict:
                val = (player,player_team, player_position, player_price, total_points)
                player_price_dict[player] = val
    GW_dict[GW] = player_price_dict
        #print("Player {} had {} price total in all {} played.".format(player, player_price, len(player_entries)))

gw_35.head()
data = gw_35[['name', 'team', 'position', 'total_points', 'value']]        
data.head()

# Helper variables
POS = data.position.unique()
CLUBS = data.team.unique()

BUDGET = 1000
pos_available = {
'DEF': 5,
'FWD': 3,
'MID': 5,
'GK': 2,
}

# Initialize Variables
solver_dict = {}
points_dict = {}
each_team = {}
budget_add = 0;
numbers = 0
for GW in GWs[:35]:
    print(GW)
    cur_player_price_dict = GW_dict[GW]
    names, teams, positions, prices, points = list(zip(*cur_player_price_dict.values()))
    players = [LpVariable("player_" + str(i), cat="Binary") for i in range(len(names))]
    # Initialize the problem
    prob = LpProblem("FPL_Player_Choices", LpMaximize)
    # Define the objective
    prob += lpSum(players[i] * points[i] for i in range(len(names))) # Objective
    # Build the constraints
    prob += lpSum(players[i] * prices[i] for i in range(len(names))) <= BUDGET # Budget Limit

    for pos in POS:
        prob += lpSum(players[i] for i in range(len(names)) if positions[i] == pos) <= pos_available[pos] # Position Limit

    for club in CLUBS:
        prob += lpSum(players[i] for i in range(len(names)) if teams[i] == club) <= 3 # Club Limit
    # Solve the problem
    prob.solve()
    for v in prob.variables():
        if v.varValue != 0:
            index = int(v.name.split("_")[1])
            name = names[index]
            club = teams[index]
            position = positions[index]
            point = points[index]
            price = prices[index]
            solver_dict[name] = (name,position,club,point,price)
            # print(name, position, club, point, price, sep=" | ")
            score = str(prob.objective)
            constraint = [str(const) for const in prob.constraints.values()][0]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraint = constraint.replace(v.name, str(v.varValue))

    score_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', score) )
    constraint_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', constraint) )

    points_dict[GW] = value(prob.objective)
    each_team[GW] = solver_dict
    solver_dict = {}
    
print(points_dict)
print(each_team) #has all 35 teams

# this function
totals_dict = [0]*35
my_team = {}
for i in range(15):
    for GW in sorted(GWs)[:35]:
        names, positions, clubs, points, prices = list(zip(*each_team[GW].values()))
        player_entries = current_df[current_df.name == names[i]]
        points = player_entries[player_entries.GW == GW]['total_points'].sum()
        totals_dict[GW-1] = points
    my_team[names[i]] = names[i], positions[i], clubs[i], prices[i], totals_dict
    totals_dict = [0]*35
    
#print(my_team)
#Solve again 4-4-2
# Helper variables
POS = data.position.unique()
CLUBS = data.team.unique()

BUDGET = 1000
pos_available = {
'DEF': [3,5], #3 to 5
'FWD': [1,3], #1 to 3
'MID': [2,5], #2 to 5
'GK': [1,1], #have to have 1 goalie (2)
} # total adds to 11

# Initialize Variables
solver_dict = {}
points_dict = {}
points_total = 0;
for GW in GWs[:35]:
    names, positions, teams, prices, points = list(zip(*my_team.values()))
    players = [LpVariable("player_" + str(i), cat="Binary") for i in range(len(names))]
 
    # Initialie the problem
    prob = LpProblem("FPL_Player_Choices", LpMaximize)
    # Define the objective
    prob += lpSum(players[i] * points[i][GW-1] for i in range(len(names))) # Objective

    for pos in POS:
        prob += lpSum(players[i] for i in range(len(names)) if positions[i] == pos) <= pos_available[pos][1]
        prob += lpSum(players[i] for i in range(len(names)) if positions[i] == pos) >= pos_available[pos][0]
    
    prob += lpSum(players[i] for i in range(len(names))) == 11
        
    # Solve the problem
    prob.solve()
    captain = 0;
    for v in prob.variables():
        if v.varValue != 0:
            index = int(v.name.split("_")[1])
            name = names[index]
            club = teams[index]
            position = positions[index]
            point = points[index][GW-1]
            if point > captain:
                captain = point
            price = prices[index]
            print(name, position, club, point, price, sep=" | ")
            score = str(prob.objective)
            constraint = [str(const) for const in prob.constraints.values()][0]
            solver_dict[name] = name, position, club, point, price
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraint = constraint.replace(v.name, str(v.varValue))

    score_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', score) )
    constraint_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', constraint) )

    #print(value(prob.objective))
    points_dict[GW] = value(prob.objective)
    points_total += value(prob.objective)
    points += captain

print(points_total)

###

# Solving for the best teams from each x week until end
for GW in sorted(GWs)[:35]:
    if GW == 1:
        current_team = each_team[GW]
    else:
        current_team = each_team[GW-1]
    print("current team")
    print(current_team)
    c_names, c_teams, c_positions, c_prices, c_points = list(zip(*current_team.values()))
    player_price_dict = {} # key: player, value: (name, team, position, price, total points)
    players = current_df[current_df.GW == GW].name.unique()
    for i, player in enumerate(players):
        player_entries = current_df[current_df.name == player]
        total_points = player_entries[player_entries.GW >= GW]['total_points'].sum()
        player_team = player_entries[player_entries.GW == GW]['team'].values[0]
        player_price = player_entries[player_entries.GW == GW]['value'].values[0]
        player_position = player_entries[player_entries.GW == GW]['position'].values[0]
        player_name = player_entries[player_entries.GW == GW]['name']
        if player not in player_price_dict:
                val = (player,player_team, player_position, player_price, total_points)
                if player in current_team:
                    c_name, c_team, c_position, c_price, c_point = current_team[player];
                    val = (player,player_team, player_position, c_price, total_points)
                player_price_dict[player] = val             
    GW_dict[GW] = player_price_dict
        #print("Player {} had {} price total in all {} played.".format(player, player_price, len(player_entries)))

        
        
# Helper variables
POS = data.position.unique()
CLUBS = data.team.unique()

BUDGET = 1000
pos_available = {
'DEF': 5,
'FWD': 3,
'MID': 5,
'GK': 2,
}

# Initialize Variables
solver_dict = {}
points_dict = {}
each_team = {}
budget_add = 0;
numbers = 0
for GW in GWs[:35]:
    print(GW)
    cur_player_price_dict = GW_dict[GW]
    names, teams, positions, prices, points = list(zip(*cur_player_price_dict.values()))
    players = [LpVariable("player_" + str(i), cat="Binary") for i in range(len(names))]
    # Initialize the problem
    prob = LpProblem("FPL_Player_Choices", LpMaximize)
    # Define the objective
    prob += lpSum(players[i] * points[i] for i in range(len(names))) # Objective
    # Build the constraints
    prob += lpSum(players[i] * prices[i] for i in range(len(names))) <= BUDGET # Budget Limit

    for pos in POS:
        prob += lpSum(players[i] for i in range(len(names)) if positions[i] == pos) <= pos_available[pos] # Position Limit

    for club in CLUBS:
        prob += lpSum(players[i] for i in range(len(names)) if teams[i] == club) <= 3 # Club Limit
    # Solve the problem
    prob.solve()
    for v in prob.variables():
        if v.varValue != 0:
            index = int(v.name.split("_")[1])
            name = names[index]
            club = teams[index]
            position = positions[index]
            point = points[index]
            price = prices[index]
            solver_dict[name] = (name,position,club,point,price)
            # print(name, position, club, point, price, sep=" | ")
            score = str(prob.objective)
            constraint = [str(const) for const in prob.constraints.values()][0]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraint = constraint.replace(v.name, str(v.varValue))

    score_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', score) )
    constraint_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', constraint) )

    points_dict[GW] = value(prob.objective)
    each_team[GW] = solver_dict
    print("i like dick")
    print(solver_dict)
    solver_dict = {}
    
print(points_dict)
print(each_team)


totals_dict = [0]*35
my_team = {}
for i in range(15):
    for GW in sorted(GWs)[:35]:
        print("poop")
        print(each_team[GW].values())
        names, positions, clubs, points, prices = list(zip(*each_team[GW].values()))
        player_entries = current_df[current_df.name == names[i]]
        points = player_entries[player_entries.GW == GW]['total_points'].sum()
        totals_dict[GW-1] = points
    my_team[names[i]] = names[i], positions[i], clubs[i], prices[i], totals_dict
    totals_dict = [0]*35

#Solve again 4-4-2
# Helper variables
POS = data.position.unique()
CLUBS = data.team.unique()

BUDGET = 1000
pos_available = {
'DEF': [3,5], #3 to 5
'FWD': [1,3], #1 to 3
'MID': [2,5], #2 to 5
'GK': [1,1], #have to have 1 goalie (2)
} # total adds to 11

# Initialize Variables
solver_dict = {}
points_dict = {}
points_total = 0;
for GW in GWs[:35]:
    names, positions, teams, prices, points = list(zip(*my_team.values()))
    players = [LpVariable("player_" + str(i), cat="Binary") for i in range(len(names))]
    #posi = [LpVariable("pos_" + str(i), cat="Integer") for i in range(4)]
 
    # Initialie the problem
    prob = LpProblem("FPL_Player_Choices", LpMaximize)
    # Define the objective
    prob += lpSum(players[i] * points[i][GW-1] for i in range(len(names))) # Objective
    
    #prob += lpSum(posi[i] for i in range(len(names))   ) == 11

    for pos in POS:
        prob += lpSum(players[i] for i in range(len(names)) if positions[i] == pos) <= pos_available[pos][1]
        prob += lpSum(players[i] for i in range(len(names)) if positions[i] == pos) >= pos_available[pos][0]
    
    prob += lpSum(players[i] for i in range(len(names))) == 11
        
    # Solve the problem
    prob.solve()
    captain = 0;
    for v in prob.variables():
        if v.varValue != 0:
            index = int(v.name.split("_")[1])
            name = names[index]
            club = teams[index]
            position = positions[index]
            point = points[index][GW-1]
            if point > captain:
                captain = point
            price = prices[index]
            print(name, position, club, point, price, sep=" | ")
            score = str(prob.objective)
            constraint = [str(const) for const in prob.constraints.values()][0]
            solver_dict[name] = name, position, club, point, price
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraint = constraint.replace(v.name, str(v.varValue))

    score_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', score) )
    constraint_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', constraint) )

    #print(value(prob.objective))
    points_dict[GW] = value(prob.objective)
    points_total += value(prob.objective)
    points += captain

print(points_total)



1
Number players in week 1 is 553
2
Number players in week 2 is 553
3
Number players in week 3 is 553
4
Number players in week 4 is 553
5
Number players in week 5 is 553
6
Number players in week 6 is 553
7
Number players in week 7 is 553
8
Number players in week 8 is 553
9
Number players in week 9 is 553
10
Number players in week 10 is 553
11
Number players in week 11 is 553
12
Number players in week 12 is 553
13
Number players in week 13 is 553
14
Number players in week 14 is 553
15
Number players in week 15 is 553
16
Number players in week 16 is 553
17
Number players in week 17 is 553
18
Number players in week 18 is 553
19
Number players in week 19 is 553
20
Number players in week 20 is 553
21
Number players in week 21 is 553
22
Number players in week 22 is 553
23
Number players in week 23 is 553
24
Number players in week 24 is 553
25
Number players in week 25 is 553
26
Number players in week 26 is 553
27
Number players in week 27 is 553
28
Number players in week 28 is 553
29
Number 

i like dick
{'Eric Dier': ('Eric Dier', 'DEF', 'Spurs', 20, 45), 'Pontus Jansson': ('Pontus Jansson', 'DEF', 'Brentford', 22, 45), 'Aymeric Laporte': ('Aymeric Laporte', 'DEF', 'Man City', 20, 55), 'Mohamed Salah': ('Mohamed Salah', 'MID', 'Liverpool', 30, 125), 'Saïd Benrahma': ('Saïd Benrahma', 'MID', 'West Ham', 26, 60), 'Gabriel Fernando de Jesus': ('Gabriel Fernando de Jesus', 'FWD', 'Man City', 24, 85), 'Dominic Calvert-Lewin': ('Dominic Calvert-Lewin', 'FWD', 'Everton', 23, 80), 'Marcos Alonso': ('Marcos Alonso', 'DEF', 'Chelsea', 23, 55), 'Hugo Lloris': ('Hugo Lloris', 'GK', 'Spurs', 27, 55), 'David Raya Martin': ('David Raya Martin', 'GK', 'Brentford', 17, 45), 'Sergio Reguilón': ('Sergio Reguilón', 'DEF', 'Spurs', 22, 50), 'Michail Antonio': ('Michail Antonio', 'FWD', 'West Ham', 40, 75), 'Ferran Torres': ('Ferran Torres', 'MID', 'Man City', 23, 70), 'Mason Greenwood': ('Mason Greenwood', 'MID', 'Man Utd', 26, 75), 'Paul Pogba': ('Paul Pogba', 'MID', 'Man Utd', 22, 75)}
4
Wel

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kylecasey/Documents/Fantasy-Premier-League/venv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/acb15f89f772467d8073b2456dfd6b53-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/acb15f89f772467d8073b2456dfd6b53-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3179 RHS
At line 3205 BOUNDS
At line 3759 ENDATA
Problem MODEL has 25 rows, 553 columns and 1659 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 592.25 - 0.00 seconds
Cgl0004I processed model has 25 rows, 365 columns (365 integer (353 of which binary)) and 1095 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.5
Cbc00

i like dick
{'João Pedro Cavaco Cancelo': ('João Pedro Cavaco Cancelo', 'DEF', 'Man City', 50, 60), 'Pontus Jansson': ('Pontus Jansson', 'DEF', 'Brentford', 40, 45), 'Mohamed Salah': ('Mohamed Salah', 'MID', 'Liverpool', 83, 125), 'Edouard Mendy': ('Edouard Mendy', 'GK', 'Chelsea', 39, 60), 'Saïd Benrahma': ('Saïd Benrahma', 'MID', 'West Ham', 47, 60), 'Ethan Pinnock': ('Ethan Pinnock', 'DEF', 'Brentford', 40, 45), 'Abdoulaye Doucouré': ('Abdoulaye Doucouré', 'MID', 'Everton', 45, 55), 'Jamie Vardy': ('Jamie Vardy', 'FWD', 'Leicester', 56, 105), 'Andros Townsend': ('Andros Townsend', 'MID', 'Everton', 45, 55), 'Marcos Alonso': ('Marcos Alonso', 'DEF', 'Chelsea', 39, 55), 'Allan Saint-Maximin': ('Allan Saint-Maximin', 'FWD', 'Newcastle', 39, 65), 'Hugo Lloris': ('Hugo Lloris', 'GK', 'Spurs', 36, 55), 'Shane Duffy': ('Shane Duffy', 'DEF', 'Brighton', 40, 40), 'Heung-Min Son': ('Heung-Min Son', 'MID', 'Spurs', 51, 100), 'Michail Antonio': ('Michail Antonio', 'FWD', 'West Ham', 50, 75)}
9


i like dick
{'João Pedro Cavaco Cancelo': ('João Pedro Cavaco Cancelo', 'DEF', 'Man City', 67, 60), 'Mohamed Salah': ('Mohamed Salah', 'MID', 'Liverpool', 117, 125), 'Edouard Mendy': ('Edouard Mendy', 'GK', 'Chelsea', 53, 60), 'Jarrod Bowen': ('Jarrod Bowen', 'MID', 'West Ham', 55, 65), 'Conor Gallagher': ('Conor Gallagher', 'MID', 'Crystal Palace', 62, 55), 'Saïd Benrahma': ('Saïd Benrahma', 'MID', 'West Ham', 57, 60), 'Antonio Rüdiger': ('Antonio Rüdiger', 'DEF', 'Chelsea', 53, 55), 'Jamie Vardy': ('Jamie Vardy', 'FWD', 'Leicester', 60, 105), 'Allan Saint-Maximin': ('Allan Saint-Maximin', 'FWD', 'Newcastle', 45, 65), 'Emile Smith Rowe': ('Emile Smith Rowe', 'MID', 'Arsenal', 57, 55), 'Tino Livramento': ('Tino Livramento', 'DEF', 'Southampton', 48, 40), 'Alex McCarthy': ('Alex McCarthy', 'GK', 'Southampton', 47, 45), 'Trent Alexander-Arnold': ('Trent Alexander-Arnold', 'DEF', 'Liverpool', 64, 75), 'Reece James': ('Reece James', 'DEF', 'Chelsea', 63, 55), 'Michail Antonio': ('Michail A

...

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kylecasey/Documents/Fantasy-Premier-League/venv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/1197d081651c4bd49fcc48f8faf7d814-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/1197d081651c4bd49fcc48f8faf7d814-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3979 RHS
At line 4005 BOUNDS
At line 4732 ENDATA
Problem MODEL has 25 rows, 726 columns and 2178 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 270.867 - 0.00 seconds
Cgl0004I processed model has 25 rows, 311 columns (311 integer (305 of which binary)) and 933 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.895105


poop
dict_values([('Martin Dubravka', 'GK', 'Newcastle', 28, 44), ('Andrew Robertson', 'DEF', 'Liverpool', 27, 73), ('Cristiano Ronaldo dos Santos Aveiro', 'FWD', 'Man Utd', 41, 122), ('Kasper Schmeichel', 'GK', 'Leicester', 24, 48), ('Bruno Guimarães Rodriguez Moura', 'MID', 'Newcastle', 34, 49), ('Heung-Min Son', 'MID', 'Spurs', 46, 177), ('Nathan Collins', 'DEF', 'Burnley', 23, 43), ('Danny Welbeck', 'FWD', 'Brighton', 23, 59), ('Marcos Alonso', 'DEF', 'Chelsea', 34, 55), ('Josh Brownhill', 'MID', 'Burnley', 28, 43), ('Gabriel Fernando de Jesus', 'FWD', 'Man City', 42, 84), ('Anthony Elanga', 'MID', 'Man Utd', 23, 48), ('Dan Burn', 'DEF', 'Newcastle', 26, 44), ('Leandro Trossard', 'MID', 'Brighton', 34, 58), ('Connor Roberts', 'DEF', 'Burnley', 24, 44)])
poop
dict_values([('Bukayo Saka', 'MID', 'Arsenal', 27, 141), ('Martin Dubravka', 'GK', 'Newcastle', 21, 44), ('Andrew Robertson', 'DEF', 'Liverpool', 27, 73), ('Cristiano Ronaldo dos Santos Aveiro', 'FWD', 'Man Utd', 40, 122), ('Br

Rúben Santos Gato Alves Dias | DEF | Man City | 2 | 61
Alex Nicolao Telles | DEF | Man Utd | 6 | 51
Yves Bissouma | MID | Brighton | 10 | 44
Raphaël Varane | DEF | Man Utd | 12 | 55
Jordan Pickford | GK | Everton | 6 | 48
Naby Keita | MID | Liverpool | 17 | 50
Heung-Min Son | MID | Spurs | 2 | 204
Jack Cork | MID | Burnley | 7 | 44
Danny Welbeck | FWD | Brighton | 2 | 59
Rob Holding | DEF | Arsenal | 3 | 41
Gabriel Magalhães | DEF | Arsenal | 2 | 54
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kylecasey/Documents/Fantasy-Premier-League/venv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/6a042e6dd33b4bc5a8cbf71eb6f9bfbf-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/6a042e6dd33b4bc5a8cbf71eb6f9bfbf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 10

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kylecasey/Documents/Fantasy-Premier-League/venv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/c9ee09127d2745c98fae1b0ef940461e-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/c9ee09127d2745c98fae1b0ef940461e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 104 RHS
At line 114 BOUNDS
At line 130 ENDATA
Problem MODEL has 9 rows, 15 columns and 45 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 91 - 0.00 seconds
Cgl0004I processed model has 4 rows, 12 columns (12 integer (11 of which binary)) and 24 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found 

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kylecasey/Documents/Fantasy-Premier-League/venv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/6c97ed5d97ee4a6a9dec5907e9be07a9-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/6c97ed5d97ee4a6a9dec5907e9be07a9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 104 RHS
At line 114 BOUNDS
At line 130 ENDATA
Problem MODEL has 9 rows, 15 columns and 45 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 79 - 0.00 seconds
Cgl0004I processed model has 4 rows, 12 columns (12 integer (11 of which binary)) and 24 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found 